# 数据准备

## 数据预处理

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'

In [2]:
from help import read_data, data_preprocess
train, test, store = read_data()
X, y, X_predict = data_preprocess(train, test, store)
del train, test, store

Loading train data...


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
//anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Loading test data...
Loading store data...
  Successfully read data as: train, test, store.

Preprocessing training data...
  Successfully preprocess training data.

Preprocessing testing data...
  Successfully preprocess testing data.



## 数据划分

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# 模型

[定义评分函数：](https://www.cnblogs.com/harvey888/p/6964741.html)

In [4]:
from help import rmspe
from sklearn.metrics import make_scorer
respe_scorer = make_scorer(rmspe, greater_is_better=False)

## 多元回归

### 线性回归

In [21]:
from sklearn.linear_model import LinearRegression
Linear_Model = LinearRegression()
Linear_Model.fit(X_train, y_train)

import joblib
FileName = 'Model_Parameter/'+'Linear_Model'+'.pkl'
joblib.dump(Model, FileName) 
# Linear_Model = joblib.load(FileName)

from help import rmspe
y_train_hat = Linear_Model.predict(X_train)
print('train rmspe score: ',rmspe(y_train_hat, y_train))
y_test_hat = Linear_Model.predict(X_test)
print('test rmspe score: ',rmspe(y_test_hat, y_test))

train rmspe score:  0.4758226182059794
test rmspe score:  0.44749689096534584


### 多项式回归

In [23]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
Ploy_Res_Model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression())])
Ploy_Res_Model.fit(X_train[:6000], y_train[:6000])

import joblib
FileName = 'Model_Parameter/'+'Ploy_Res_Model'+'.pkl'
joblib.dump(Model, FileName) 

['Model_Parameter/Ploy_Res_Model.pkl']

In [24]:
from help import rmspe
y_train_hat = Ploy_Res_Model.predict(X_train[:6000])
print('train rmspe score: ',rmspe(y_train_hat, y_train[:6000]))
y_test_hat = Ploy_Res_Model.predict(X_test)
print('test rmspe score: ',rmspe(y_test_hat, y_test))

train rmspe score:  0.33897864536219596
test rmspe score:  9909741421.722847


## SVR

### LinearSVR

In [25]:
from sklearn.svm import LinearSVR
Linear_SVR = LinearSVR()
Linear_SVR.fit(X_train, y_train)

import joblib
FileName = 'Model_Parameter/'+'Linear_SVR'+'.pkl'
joblib.dump(Model, FileName) 

from help import rmspe
y_train_hat = Linear_SVR.predict(X_train)
y_test_hat = Linear_SVR.predict(X_test)
print('train rmspe score: ',rmspe(y_train_hat, y_train))
print('test rmspe score: ',rmspe(y_test_hat, y_test))

train rmspe score:  0.4337112882677548
test rmspe score:  0.40784632317003394


In [26]:
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'epsilon': [0, 0.01, 0.1, 1, 10],
              'max_iter': [1000]}

Linear_SVR_grid_search = GridSearchCV(LinearSVR(), param_grid, cv=5, 
                                      n_jobs=-1, return_train_score=True,
                                     scoring=respe_scorer)
Linear_SVR_grid_search.fit(X_train[:10000], y_train[:10000])

import joblib
FileName = 'Model_Parameter/'+'Linear_SVR_grid_search'+'.pkl'
joblib.dump(Model, FileName) 

['Model_Parameter/Linear_SVR_grid_search.pkl']

In [27]:
Linear_SVR_grid_search.best_estimator_

LinearSVR(C=1000, dual=True, epsilon=0.1, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [28]:
from help import rmspe
y_train_hat = Linear_SVR_grid_search.predict(X_train[:10000])
print('train rmspe score: ',rmspe(y_train_hat, y_train[:10000]))
y_test_hat = Linear_SVR_grid_search.predict(X_test[:10000])
print('test rmspe score: ',rmspe(y_test_hat, y_test[:10000]))

train rmspe score:  0.40222952160414394
test rmspe score:  0.4086374977505892


### KernelSVR

In [30]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['rbf', 'poly'],
              'C': [1],
              'epsilon': [0, 0.01, 0.1, 1, 10],
              'degree': [1, 3, 5, 7],
              'gamma': ['auto'],
              'max_iter': [-1]}

SVR_grid_search = GridSearchCV(SVR(), param_grid, cv=2, 
                               n_jobs=-1, return_train_score=True,
                              scoring=respe_scorer)
SVR_grid_search.fit(X_train[:6000], y_train[:6000])

print(SVR_grid_search)
import joblib
FileName = 'Model_Parameter/'+'SVR_grid_search'+'.pkl'
joblib.dump(Model, FileName) 

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [1], 'degree': [1, 3, 5, 7],
                         'epsilon': [0, 0.01, 0.1, 1, 10], 'gamma': ['auto'],
                         'kernel': ['rbf', 'poly'], 'max_iter': [-1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=make_scorer(rmspe, greater_is_better=False), verbose=0)


['Model_Parameter/SVR_grid_search.pkl']

In [31]:
SVR_grid_search.best_estimator_

SVR(C=1, cache_size=200, coef0=0.0, degree=1, epsilon=0, gamma='auto',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [32]:
from help import rmspe
y_train_hat = SVR_grid_search.predict(X_train[:6000])
print('train rmspe score: ',rmspe(y_train_hat, y_train[:6000]))
y_test_hat = SVR_grid_search.predict(X_test[:6000])
print('test rmspe score: ',rmspe(y_test_hat, y_test[:6000]))

train rmspe score:  0.42596767822063136
test rmspe score:  0.4358476143737631


## 决策树

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier(random_state=42)

# 生成参数列表
parameters = {'max_depth':[2,4,6,8,10],'min_samples_leaf':[2,4,6,8,10], 'min_samples_split':[2,4,6,8,10]}

# 定义计分器
scorer = make_scorer(f1_score)

# 生成网络搜索器
grid_obj = GridSearchCV(clf, parameters, scoring=scorer)

# 拟合网络搜索器
grid_fit = grid_obj.fit(X_train, y_train)